In [ ]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from scipy import stats
from statistics import mean, stdev
from math import sqrt
from scipy.stats import bartlett
import pandas as pd
import pickle

game_data = (r"game_data")
# import the datasets
with open("Pickles/Datasets.pkl", "rb") as l:
    datasets = pickle.load(l)


wobu_dict = {}                                   # one dictionary for each team
fck_dict = {}                                    # Wolfsburg and Köln

for dataset in datasets:                         # in both dictionaries one list per dataset (datasets; area & interval)
    wobu_dict[dataset] = []
    fck_dict[dataset]= []
    
    
# wobu_dict[datasets[0]] = []                      
# wobu_dict[datasets[1]] = []
# wobu_dict[datasets[2]] = []
# wobu_dict[datasets[3]] = []
# fck_dict[datasets[0]] = []
# fck_dict[datasets[1]] = []
# fck_dict[datasets[2]] = []
# fck_dict[datasets[3]] = []


for dataset in datasets:                         # for every dataset
    for dir in os.listdir(game_data):               # for every directory (=folder per game) in game_data directory
        loc = os.path.join(game_data, dir)           # saving this game folder as loc
        # os.chdir(loc)
        for file in os.listdir(loc):                    # for every file in loc (all the datasets for one game)
            if file[-26:-17] == dataset and file[9:12] == "koe" and file.startswith("zero"): #current dataset (a & i), if koeln was hometeam & and offset file
                #print(file)
                with open(os.path.join(loc, file), "r") as f:                  # open file
                    reader = csv.reader(f, delimiter=',')                      # read file
                    for row in reader:                                         # for every row in file
                        if row != []:                                          # if file is not empty 
                            fck_dict[dataset].append([float(row[-1]), 0, file[9:], row[0], row[1]])              # TEAM B (=away = Opponent): Success Score, 0, file name without offset and game code, minute & second
                            fck_dict[dataset].append([float(row[-3]), 1, file[9:], row[0], row[1]])              # TEAM A (=home = Köln): Success Score, 1, file name without offset and game code, minute & second
            elif file[-26:-17] == dataset and file[-41:-38] == "koe" and file.startswith("zero"): #current dataset (a & i), if koeln was away team & and offset file
                #print(file)
                with open(os.path.join(loc, file), "r") as f:
                    reader = csv.reader(f, delimiter=',')
                    for row in reader:
                        if row != []:
                            fck_dict[dataset].append([float(row[-1]), 1, file[9:], row[0], row[1]])              # TEAM B (=away = Köln): Success Score, 1, file name without offset and game code, minute & second
                            fck_dict[dataset].append([float(row[-3]), 0, file[9:], row[0], row[1]])              # TEAM A (=home = Opponent): Success Score, 0, file name without offset and game code, minute & second
            if file[-26:-17] == dataset and file[9:12] == "wol" and file.startswith("zero"): # current dataset (a & i), if Wolfsburg was home team & and offset file
                #print(file)
                with open(os.path.join(loc, file), "r") as f:
                    reader = csv.reader(f, delimiter=',')
                    for row in reader:
                        if row != []:
                            wobu_dict[dataset].append([float(row[-1]), 0, file[9:], row[0], row[1]])             # TEAM B (=away = Opponent): Success Score, 0, file name without offset and game code, minute & second
                            wobu_dict[dataset].append([float(row[-3]), 1, file[9:], row[0], row[1]])             # TEAM A (=home = Wolfsburg): Success Score, 1, file name without offset and game code, minute & second
            elif file[-26:-17] == dataset and file[-41:-38] == "wol" and file.startswith("zero"): #if wolfsburg was awayteam
                #print(file)
                with open(os.path.join(loc, file), "r") as f:
                    reader = csv.reader(f, delimiter=',')
                    for row in reader:
                        if row != []:
                            wobu_dict[dataset].append([float(row[-1]), 1, file[9:], row[0], row[1]])             # TEAM B (=away = Wolfsburg): Success Score, 1, file name without offset and game code, minute & second
                            wobu_dict[dataset].append([float(row[-3]), 0, file[9:], row[0], row[1]])             # TEAM A (=home = Opponent): Success Score, 0, file name without offset and game code, minute & second

                            # dictionaries for success scores for both teams                            
koeln_scores = {}
wolfsburg_scores = {}
for dataset in datasets:               # in those dictionaries one list per dataset (a&i)
    koeln_scores[dataset] = []
    wolfsburg_scores[dataset] = []

    
for dataset in datasets:              # for every dataset (a&i)
    for datapoint in fck_dict[dataset]:        # for every frame in that list in the dictionary
        if datapoint[1] == 1:                  # frame of Köln (=! Opponent)?
            koeln_scores[dataset].append(float(datapoint[0])) # then append the success score
    for datapoint in wobu_dict[dataset]:
        if datapoint[1] == 1:                  # frame of Wolfsburg (=! Opponent)?
            wolfsburg_scores[dataset].append(float(datapoint[0])) # then append the success score

            
# same as above but for Köln's and Wolfsburg's opponents                          
koeln_scores_o = {}
wolfsburg_scores_o = {}
for dataset in datasets:               
    koeln_scores_o[dataset] = []
    wolfsburg_scores_o[dataset] = []

    
for dataset in datasets:              
    for datapoint in fck_dict[dataset]:        
        if datapoint[1] == 0:                  # frame of Opponent (=! Köln)?
            koeln_scores_o[dataset].append(float(datapoint[0])) # then append the success score
    for datapoint in wobu_dict[dataset]:
        if datapoint[1] == 0:                  # frame of Opponent (!= Wolfsburg)?
            wolfsburg_scores_o[dataset].append(float(datapoint[0])) # then append the success score


fck_wolf = {}
i = 0
for dataset in datasets:  # for every dataset (a&i)
    fck_wolf[dataset]=[]
    i += 1
    koeln_scores[dataset] = np.array(koeln_scores[dataset])            # list to array
    fck_wolf[dataset].append(round(mean(koeln_scores[dataset]), 2))
    fck_wolf[dataset].append(round(mean(wolfsburg_scores[dataset]),2))
    

i = 0
opponents ={}
for dataset in datasets:  # for every dataset (a&i)
    i += 1
    opponents[dataset] = []
    koeln_scores_o[dataset] = np.array(koeln_scores_o[dataset])
    opponents[dataset].append(round(mean(koeln_scores_o[dataset]), 2))    
    opponents[dataset].append(round(mean(wolfsburg_scores_o[dataset]), 2))    
    

In [ ]:
fck_wolf

In [ ]:
opponents

In [ ]:
with open("Pickles/koeln_wolfsburg.pkl", "wb") as cdict:
    pickle.dump(fck_wolf, cdict)
    
with open("Pickles/opponents.pkl", "wb") as cdict:
    pickle.dump(opponents, cdict)
   
    
with open("Pickles/wolfsburg_scores.pkl", "wb") as cdict:
    pickle.dump(wolfsburg_scores, cdict)
    
with open("Pickles/wolfsburg_scores_opp.pkl", "wb") as cdict:
    pickle.dump(wolfsburg_scores_o, cdict)
    
with open("Pickles/koeln_scores.pkl", "wb") as cdict:
    pickle.dump(koeln_scores_o, cdict)
    
with open("Pickles/koeln_scores_opp.pkl", "wb") as cdict:
    pickle.dump(koeln_scores_o, cdict)

# MWU
## Köln vs Wolfsburg

In [ ]:
import pandas as pd
MWU = pd.DataFrame(index=['U', 'p', 'Effektstärke'])
for d in datasets:
    print(d)
    result = stats.mannwhitneyu(x=koeln_scores[d], y=wolfsburg_scores[d], alternative = 'two-sided')
    print(result)
    Effekt = result[0]/(len(koeln_scores[d])*len(wolfsburg_scores[d]))
    print(Effekt)
    MWU[d] = [result[0], result[1], Effekt]

In [ ]:
MWU.to_csv("Results/MWU.csv")
MWU

In [ ]:
MWU16 = MWU.iloc[:, 0:17]
names16 = []
for name in MWU16.columns:
    a = name.split('_')[0]
    b = name.split('_')[1]
    c = ' '.join([a, b])
    names16.append(c)
    
MWU30 = MWU.iloc[:,17:]
names30 = []
for name in MWU30.columns:
    a = name.split('_')[0]
    b = name.split('_')[1]
    c = ' '.join([a, b])
    names30.append(c)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(16,9), gridspec_kw={'height_ratios': [4, 1]})


fig.subplots_adjust(hspace=0.05)  # adjust space between axes

ax1.scatter(x= list(MWU16.columns), y= MWU16.iloc[2], c='#005270')
ax1.plot(list(MWU16.columns), MWU16.iloc[2], c='#005270', linestyle='dotted')
ax1.hlines(y=0.5, xmin = 0, xmax = 16, linestyles='dashed', color='black')

ax2.scatter(x= list(MWU16.columns), y= MWU16.iloc[2], c='#005270')
ax2.plot(list(MWU16.columns), MWU16.iloc[2], c='#005270', linestyle='dotted', linewidth = 1.5)

ax2.set_ylim(0, 0.05) 
ax2.set_yticks(np.arange(0, 0.051, 0.025))
ax1.set_ylim(0.2, 0.55)  # most of the data
ax1.spines.bottom.set_visible(False)
ax2.spines.top.set_visible(False)
#ax1.set_xticks([])
ax1.xaxis.tick_top()
ax1.tick_params(axis='x',labeltop=False, length=0)  # don't put tick labels at the top
ax2.xaxis.tick_bottom()
ax2.set_xticklabels(names16, rotation = 45)

d = 0.5  # proportion of vertical to horizontal extent of the slanted line
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
ax1.plot([0, 1], [0, 0], transform=ax1.transAxes, **kwargs)
ax2.plot([0, 1], [1, 1], transform=ax2.transAxes, **kwargs)
plt.savefig('Results/MWU16.png',facecolor ="w",
            edgecolor ='w', bbox_inches='tight', dpi=500)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(16,9), gridspec_kw={'height_ratios': [4, 1]})


fig.subplots_adjust(hspace=0.05)  # adjust space between axes

ax1.scatter(x= list(MWU30.columns), y= MWU30.iloc[2], c='#005270')
ax1.plot(list(MWU30.columns), MWU30.iloc[2], c='#005270', linestyle='dotted')
ax1.hlines(y=0.5, xmin = 0, xmax = 15, linestyles='dashed', color='black', linewidth = 1.5)

ax2.scatter(x= list(MWU30.columns), y= MWU30.iloc[2], c='#005270')
ax2.plot(list(MWU30.columns), MWU30.iloc[2], c='#005270', linestyle='dotted')

ax2.set_ylim(0, 0.05) 
ax2.set_yticks(np.arange(0, 0.051, 0.025))
ax1.set_ylim(0.2, 0.55)  # most of the data
ax1.spines.bottom.set_visible(False)
ax2.spines.top.set_visible(False)
#ax1.set_xticks([])
ax1.xaxis.tick_top()
ax1.tick_params(axis='x',labeltop=False, length=0)  # don't put tick labels at the top
ax2.xaxis.tick_bottom()
ax2.set_xticklabels(names30, rotation = 45)

d = 0.5  # proportion of vertical to horizontal extent of the slanted line
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
ax1.plot([0, 1], [0, 0], transform=ax1.transAxes, **kwargs)
ax2.plot([0, 1], [1, 1], transform=ax2.transAxes, **kwargs)
plt.savefig('Results/MWU30.png',facecolor ="w",
            edgecolor ='w', bbox_inches='tight', dpi=500)

## Gegner

In [ ]:
import pandas as pd
MWU_o = pd.DataFrame(index=['U', 'p', 'Effektstärke'])
for d in datasets:
    print(d)
    result = stats.mannwhitneyu(x=koeln_scores_o[d], y=wolfsburg_scores_o[d], alternative = 'two-sided')
    print(result)
    Effekt = result[0]/(len(koeln_scores_o[d])*len(wolfsburg_scores_o[d]))
    print(Effekt)
    MWU_o[d] = [result[0], result[1], Effekt]

In [ ]:
MWU_o.to_csv("Results/MWU_o.csv")
MWU_o

In [ ]:
MWU_o16 = MWU_o.iloc[:, 0:17]
names16 = []
for name in MWU_o16.columns:
    a = name.split('_')[0]
    b = name.split('_')[1]
    c = ' '.join([a, b])
    names16.append(c)
    
MWU_o30 = MWU_o.iloc[:,17:]
names30 = []
for name in MWU_o30.columns:
    a = name.split('_')[0]
    b = name.split('_')[1]
    c = ' '.join([a, b])
    names30.append(c)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(16,9), gridspec_kw={'height_ratios': [4, 1]})


fig.subplots_adjust(hspace=0.05)  # adjust space between axes

ax1.scatter(x= list(MWU_o16.columns), y= MWU_o16.iloc[2], c='#005270')
ax1.plot(list(MWU_o16.columns), MWU_o16.iloc[2], c='#005270', linestyle='dotted')
ax1.hlines(y=0.5, xmin = 0, xmax = 16, linestyles='dashed', color='black',  linewidth = 2.5)

ax2.scatter(x= list(MWU_o16.columns), y= MWU_o16.iloc[2], c='#005270')
ax2.plot(list(MWU_o16.columns), MWU_o16.iloc[2], c='#005270', linestyle='dotted')

ax2.set_ylim(0, 0.05) 
ax2.set_yticks(np.arange(0, 0.051, 0.025))
ax1.set_ylim(0.5, 0.8)  # most of the data
ax1.spines.bottom.set_visible(False)
ax2.spines.top.set_visible(False)
#ax1.set_xticks([])
ax1.xaxis.tick_top()
ax1.tick_params(axis='x',labeltop=False, length=0)  # don't put tick labels at the top
ax2.xaxis.tick_bottom()
ax2.set_xticklabels(names16, rotation = 45)

d = 0.5  # proportion of vertical to horizontal extent of the slanted line
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
ax1.plot([0, 1], [0, 0], transform=ax1.transAxes, **kwargs)
ax2.plot([0, 1], [1, 1], transform=ax2.transAxes, **kwargs)
plt.savefig('Results/MWU_o16.png',facecolor ="w",
            edgecolor ='w', bbox_inches='tight', dpi=500)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(16,9), gridspec_kw={'height_ratios': [4, 1]})


fig.subplots_adjust(hspace=0.05)  # adjust space between axes

ax1.scatter(x= list(MWU_o30.columns), y= MWU_o30.iloc[2], c='#005270')
ax1.plot(list(MWU_o30.columns), MWU_o30.iloc[2], c='#005270', linestyle='dotted')
ax1.hlines(y=0.5, xmin = 0, xmax = 15, linestyles='dashed', color='black', linewidth = 2.5)


ax2.scatter(x= list(MWU_o30.columns), y= MWU_o30.iloc[2], c='#005270')
ax2.plot(list(MWU_o30.columns), MWU_o30.iloc[2], c='#005270', linestyle='dotted')

ax2.set_ylim(0, 0.05) 
ax2.set_yticks(np.arange(0, 0.051, 0.025))
ax1.set_ylim(0.5, 0.8)  # most of the data
ax1.spines.bottom.set_visible(False)
ax2.spines.top.set_visible(False)
#ax1.set_xticks([])
ax1.xaxis.tick_top()
ax1.tick_params(axis='x',labeltop=False, length=0)  # don't put tick labels at the top
ax2.xaxis.tick_bottom()
ax2.set_xticklabels(names30, rotation = 45)

d = 0.5  # proportion of vertical to horizontal extent of the slanted line
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
ax1.plot([0, 1], [0, 0], transform=ax1.transAxes, **kwargs)
ax2.plot([0, 1], [1, 1], transform=ax2.transAxes, **kwargs)
plt.savefig('Results/MWU_o30.png',facecolor ="w",
            edgecolor ='w', bbox_inches='tight', dpi=500)